In [9]:
import pandas as pd
import os

In [10]:
directory_path = 'C:\Users\abhie\OneDrive - Northeastern University\ALY6980\ipedsllm\data_ingest\ipeds\dictionaries'

In [11]:
table_names = ["ADM2022", "C2022DEP", "EFFY2022", "GR2022", "HD2022", "IC2022CAMPUSES"]

In [12]:
ADM2022_prefix = ""
C2022DEP_prefix = ""
EFFY2022_prefix =""
HD2022_prefix = ""
GR2022_prefix = ""
IC2022_CAMPUSES_prefix = ""

In [13]:
count = 0
for filename in os.listdir(directory_path):
    # Check if the item is a file (not a subdirectory)
    if os.path.isfile(os.path.join(directory_path, filename)):
        df = pd.read_excel(directory_path+"/"+filename,sheet_name = 'varlist')
        df1 = pd.read_excel(directory_path+"/"+filename,sheet_name = 'Frequencies')
        df2 = pd.read_excel(directory_path+"/"+filename,sheet_name = 'Introduction')
        dataset = df[['varname','format','DataType','varTitle']]
        dataset2 = df1[['varname','codevalue','valuelabel']]
        dataset.loc[:, 'DataType'] = dataset['DataType'].replace({'N': 'Numeric', 'A': 'Text'})
        tabledesc = str(str(df2['TableDescription'].iloc[0]).replace(',', ';').replace(':', '').replace('/',' or '))
        tableName = str(str(df2['TableName'].iloc[0]))
        del df, df1
        if table_names[count].lower() in filename:
            descr = ""
            for index, row in dataset.iterrows():
                varName= ''
                desc = ''
                if row['format'] in 'Disc':
                    varName = str(row['varname']).lower()
                    desc = str(row['varTitle']).replace(',', ';').replace(':', '').replace('/',' or ')
                    dtype = str(row['DataType'])
                    values = {}
                    for index2, row2 in dataset2.iterrows():
                        if row['varname'] in row2['varname']:
                            values[str(row2['codevalue'])] = str(row2['valuelabel']).replace(',', ';').replace(':', '').replace("\\",'').replace("'","").replace('/',' or ')
                            
                    if len(descr) == 0:
                        descr += '{ "Table_Info": [ { "Table_Name": "'+ tableName + '", "Table_Description": '+ tabledesc +', "Columns": [ { "Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc +'", "encoded_values": "' + str(values) +'" }'
                    else:
                        descr += ', {"Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc  + '", "encoded_values": "' + str(values) +'" }'
                else:
                    varName = str(row['varname']).lower()
                    desc = str(row['varTitle']).replace(',', ';').replace(':', '').replace('/',' or ')
                    dtype = str(row['DataType'])
                    if len(descr) == 0:
                        descr += '{ "Table_Info": [ {"Table_Name": "'+ tableName + '", "Table_Description": '+ tabledesc +', "Columns": [ { "Column_Name": "' + varName + '", "Data_Type": "'+ dtype+ '", "Column_Description": "'+ desc+'" }'
                    else:
                        descr += ', {"Column_Name": "'+ varName + '", "Data_Type": "'+ dtype + '", "Column_Description": "'+ desc +'" }'
            descr += ']}]}'
            if count == 0:
                ADM2022_prefix += descr
            elif count ==1:
                C2022DEP_prefix += descr
            elif count ==2:
                EFFY2022_prefix += descr
            elif count ==3:
                GR2022_prefix += descr
            elif count ==4:
                HD2022_prefix += descr
            else:
                IC2022_CAMPUSES_prefix += descr
    count+=1

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data_ingest\\ipeds\\dictionaries'

In [ ]:
# count = 0
# for filename in os.listdir(directory_path):
#     # Check if the item is a file (not a subdirectory)
#     if os.path.isfile(os.path.join(directory_path, filename)):
#         df = pd.read_excel(directory_path+"/"+filename,sheet_name = 'varlist')
#         df1 = pd.read_excel(directory_path+"/"+filename,sheet_name = 'Frequencies')
#         dataset = df[['varname','format','varTitle']]
#         dataset2 = df1[['varname','codevalue','valuelabel']]
#         del df, df1
#         if table_names[count].lower() in filename:
#             descr = ""
#             vals =  ""
#             for index, row in dataset.iterrows():
#                 varName= ''
#                 desc = ''
#                 if row['format'] in 'Disc':
#                     varName = str(row['varname']).lower()
#                     desc = str(row['varTitle']).replace(',', '').replace(':', '').replace('/',' or ')
#                     if descr in '':
#                         descr += "SQL Table "+ table_names[count] + " contains {" + varName + ": "+ desc
#                     else:
#                         descr += ", "+ varName + ": "+ desc
#                     values = {}
#                     for index2, row2 in dataset2.iterrows():
#                         if row['varname'] in row2['varname']:
#                             values[row2['codevalue']] = str(row2['valuelabel']).replace(',', '').replace(':', '')
#                     vals += ". Variable " + varName + " contains " + str(values)
#                 else:
#                     varName = str(row['varname']).lower()
#                     desc = str(row['varTitle']).replace(',', '').replace(':', '').replace('/',' or ')
#                     if descr in '':
#                         descr += "SQL Table "+ table_names[count] + " contains {" + varName + ": "+ desc
#                     else:
#                         descr += ", "+ varName + ": "+ desc
#             descr += "}"
#             if count == 0:
#                 ADM2022_prefix += descr + vals
#             elif count ==1:
#                 C2022DEP_prefix += descr + vals
#             elif count ==2:
#                 EFFY2022_prefix += descr + vals
#             elif count ==3:
#                 GR2022_prefix += descr + vals
#             elif count ==4:
#                 HD2022_prefix += descr + vals
#             else:
#                 IC2022_CAMPUSES_prefix += descr + vals
#     count+=1

In [6]:
json_objects = [
    ADM2022_prefix,
C2022DEP_prefix,
EFFY2022_prefix,
HD2022_prefix,
GR2022_prefix,
IC2022_CAMPUSES_prefix,
]

In [7]:
import json
ls = []
for i in json_objects:
    ls.append(json.loads(i))

In [8]:
with open("tableinfo.json", "w") as json_file:
    json.dump(ls, json_file, indent=4)